In [53]:
# Import WebSocket client library
from websocket import create_connection
import threading
import json
import pandas as pd
import websocket
import datetime
from functools import partial
import timeit

In [7]:
class KrakenWebSocketClient:
    """
    Kraken Client Interface

    """

    def __init__(self, data_class ):
        websocket.enableTrace(False)
        self.data_class=data_class
        ws = websocket.WebSocketApp("wss://ws.kraken.com/",
                                         on_message=self.on_message,
                                         on_error=self.on_error,
                                         on_close=self.on_close)
        self.ws = ws
        self.ws.on_open = self.on_open
        self.ws.run_forever()

    def on_message(self,ws, message):
        if "event" not in message: 
            formated_point = json.loads(message)
#             print('WebSocket: '+formated_point[1][0]+' '+ formated_point[1][1]+' '+formated_point[1][2]+' '+str(formated_point[1][-1]))
            self.data_class.to_data_que(formated_point[1])
            
        return message

    def on_error(self,ws, error):
        print(error)
        return error

    def on_close(self, a,b,c):
        self.ws.close()
        print("### closed ###")   
        # this is where the final data will be pushed to the dbs or something 

    def run(self, *args):
        global driver
        driver = True
        while driver:
            try:
                time.sleep(1)
#                 I think we can use the below part to stop the code 
#                 p = input()
#                 self.ws.send(p)
            except KeyboardInterrupt:
                driver = False
        time.sleep(1)
        self.ws.close()
        print("thread terminating...")

    def on_open(self,ws):
        self.ws.send('{"event":"subscribe", "subscription":{"name":"ohlc"}, "pair":["XBT/USD"]}')
        t = threading.Thread(target=self.run)
        t.start()
        



In [18]:
class BTC_Data(): 
    def __init__(self):
        self.data_que=[]
        self.one_min =[]
        
    def to_data_que(self, data_point: list):
        if len(self.data_que)==0:                      # Empty que just add the point 
            self.data_que.append(data_point)

        else: 
            if data_point[1] == self.data_que[-1][1]: # In the Same interval: add the point
                self.data_que.append(data_point)
                
            else: 
                # pop the last point                 
                # Add to the 1 min data frame 
                new_point = self.data_que.pop()
                self.to_one_min(new_point)
                # clear the que 
                # add the new point 
                self.data_que.clear()
                self.data_que.append(data_point)
                
                
    def to_one_min(self, data_point:list):
        formated_point = [
            int(float(data_point[1])),
            datetime.datetime.utcfromtimestamp(int(float(data_point[1]))),
            'BTCUSD',
            float(data_point[2]),
            float(data_point[3]),
            float(data_point[4]),
            float(data_point[5]),
            float(data_point[7])    
        ]
        self.one_min.append(formated_point)
        if len(self.one_min) == 3: 
            print('1 min list')
            for x in self.one_min:
                print(x)
        
        
# 0 time	decimal	Begin time of interval, in seconds since epoch
# 1 etime	decimal	End time of interval, in seconds since epoch
# 2 open	decimal	Open price of interval
# 3 high	decimal	High price within interval
# 4 low	decimal	Low price within interval
# 5 close	decimal	Close price of interval
# 6 vwap	decimal	Volume weighted average price within interval
# 7 volume	decimal	Accumulated volume within interval
# 8 count

#     unix	date	symbol	open	high	low	close	Volume BTC
        
    


In [19]:
# USE THIS FOR TESTING IN JUYPTER NOTEBOOKS. USE THE BELOW CELL IN FOR .PY VCODE PART ONCE WE MERGE IT ALL
BTC =BTC_Data()
kraken_client = KrakenWebSocketClient(BTC)

WebSocket: 1638764618.053039 1638764640.000000 49248.80000 2
WebSocket: 1638764636.724970 1638764640.000000 49248.80000 3
WebSocket: 1638764638.333766 1638764640.000000 49248.80000 4
WebSocket: 1638764639.064524 1638764640.000000 49248.80000 5
WebSocket: 1638764639.940551 1638764640.000000 49248.80000 6
WebSocket: 1638764658.461778 1638764700.000000 49226.50000 2
WebSocket: 1638764686.377833 1638764700.000000 49226.50000 7
WebSocket: 1638764699.664535 1638764700.000000 49226.50000 8
WebSocket: 1638764703.712923 1638764760.000000 49233.30000 1
WebSocket: 1638764704.016868 1638764760.000000 49233.30000 3
WebSocket: 1638764704.886977 1638764760.000000 49233.30000 4
WebSocket: 1638764709.604577 1638764760.000000 49233.30000 5
WebSocket: 1638764716.341867 1638764760.000000 49233.30000 9
WebSocket: 1638764720.265167 1638764760.000000 49233.30000 10
WebSocket: 1638764735.545732 1638764760.000000 49233.30000 11
WebSocket: 1638764739.243119 1638764760.000000 49233.30000 13
WebSocket: 1638764752

In [ ]:
def main():
    BTC =BTC_Data()
    kraken_client = KrakenWebSocketClient(BTC)


# if __name__ == '__main__':
#     main()

In [ ]:
# Everything above this is working   # below is a work in progress

In [ ]:
## Pulling in gap data from last point in df

In [26]:
MasterData = pd.read_csv('./src/Master_Data_File.csv')

In [27]:
MasterData.iloc[-1]

Unix Timestamp             1634429100
Date              2021-10-17 00:05:00
Symbol                         BTCUSD
Open                         60716.28
High                         60716.28
Low                          60712.32
Close                        60715.55
Volume                       0.022586
Name: 3168005, dtype: object

In [36]:
# https://api.kraken.com/0/public/OHLC?pair=XBTUSD?interval=1?since=1634429100
datetime.datetime.utcfromtimestamp(1638722460)

datetime.datetime(2021, 12, 5, 16, 41)

In [80]:
# on this function need to fix function call to combine dataframes 


# used to get the trade data but only returns last 1000 points of data from the date given 
def nano_after_epoch(t):
    """Converts DateTime object to int nanoseconds after epoch"""
    return int(t.replace(tzinfo=datetime.timezone.utc).timestamp() * 1000000000)



start_time = 1634429100 * 1000000000
end_time = nano_after_epoch(datetime.datetime.utcnow())

leeway = 5*60 * 1000000000 # min* Seconds(60) * nanoseconds converter

print(start_time,end_time,leeway)


start = timeit.timeit()
dfList = []
# while start_time < end_time - leeway:
for x in range(5):
    print(f"Retrieving data from {datetime.datetime.utcfromtimestamp(start_time // 1000000000)}...")

    query_string = 'https://api.kraken.com/0/public/Trades?pair=xbtusd&since='+ str(start_time)
    resp = requests.get(query_string).json()

    # Append new data to dataframe
    
    if len(dfList)<0:
        dfList = resp['result']['XXBTZUSD']
    else: 
        dfList.append(resp['result']['XXBTZUSD'])

    # Determine last retrieved date
    start_time = int(resp['result']['last'])

    # # New 'retrieve from' date is latest_time + 1 nanosecond    # dont think ill need but maybe 
    #     start_time = nano_after_epoch(start_time) + 1

    # Sleep to avoid being rate limited
    time.sleep(1.8)

end = timeit.timeit()
print(end - start)

1634429100000000000 1638770462182164992 300000000000
Retrieving data from 2021-10-17 00:05:00...
Retrieving data from 2021-10-17 00:35:10...
Retrieving data from 2021-10-17 01:38:27...
Retrieving data from 2021-10-17 03:15:56...
Retrieving data from 2021-10-17 04:26:28...
0.0027801500000350643


In [81]:
df = pd.DataFrame(dfList)
df

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,"[60752.80000, 0.00240219, 1634429100.1276, s, ...","[60751.20000, 0.00113202, 1634429100.1287, s, ...","[60751.20000, 0.00060000, 1634429114.6284, s, ...","[60750.40000, 0.00333289, 1634429116.0743, b, ...","[60750.40000, 0.00100100, 1634429117.0587, b, ...","[60750.30000, 0.00015499, 1634429119.6577, s, ...","[60750.30000, 0.00790151, 1634429119.8435, s, ...","[60750.00000, 0.01209849, 1634429119.8445, s, ...","[60750.00000, 0.00290151, 1634429146.4614, s, ...","[60748.80000, 0.00493837, 1634429146.4628, s, ...",...,"[60658.10000, 0.04330133, 1634430910.2482, b, ...","[60658.10000, 0.24679906, 1634430910.337, b, m, ]","[60658.10000, 0.34666319, 1634430910.3894, b, ...","[60659.90000, 0.23601414, 1634430910.531, b, l, ]","[60662.70000, 0.01565501, 1634430910.5322, b, ...","[60669.30000, 0.00383085, 1634430910.5341, b, ...","[60669.30000, 0.00100000, 1634430910.8399, b, ...","[60668.00000, 0.15452733, 1634430910.9754, b, ...","[60669.30000, 0.33726883, 1634430910.9763, b, ...","[60675.90000, 0.12347653, 1634430910.9774, b, ..."
1,"[60675.90000, 0.08590626, 1634430910.9783, b, ...","[60675.90000, 0.00100000, 1634430910.9792, b, ...","[60675.90000, 0.02071928, 1634430910.9801, b, ...","[60675.90000, 0.22075575, 1634430910.9811, b, ...","[60675.90000, 0.09170930, 1634430911.0219, b, ...","[60675.90000, 0.08962544, 1634430911.1409, b, ...","[60675.90000, 0.66184316, 1634430911.1568, b, ...","[60675.90000, 0.03790000, 1634430911.219, b, l, ]","[60675.90000, 0.07683119, 1634430911.2379, b, ...","[60676.20000, 0.02238200, 1634430911.2388, b, ...",...,"[61269.40000, 0.46220081, 1634434638.325, b, l, ]","[61261.20000, 1.39340000, 1634434645.2291, s, ...","[61252.00000, 0.17090185, 1634434651.3477, s, ...","[61251.90000, 0.34292332, 1634434651.3487, s, ...","[61247.40000, 0.02937974, 1634434687.796, s, m, ]","[61249.10000, 0.03359824, 1634434707.5101, s, ...","[61249.20000, 0.00030000, 1634434707.6093, b, ...","[61249.20000, 0.00400914, 1634434707.6111, b, ...","[61249.30000, 0.00060000, 1634434707.6126, b, ...","[61249.30000, 0.00030000, 1634434707.6138, b, ..."
2,"[61256.70000, 0.00479086, 1634434707.615, b, m, ]","[61256.70000, 0.00750000, 1634434707.6167, b, ...","[61249.10000, 0.00060000, 1634434744.3296, s, ...","[61249.20000, 0.00030000, 1634434745.6421, b, ...","[61249.20000, 0.09970000, 1634434745.6434, b, ...","[61249.10000, 0.00554366, 1634434760.4913, s, ...","[61236.90000, 0.09445634, 1634434760.4926, s, ...","[61253.20000, 0.00505000, 1634434776.9571, b, ...","[61236.60000, 0.00100000, 1634434783.8669, s, ...","[61236.60000, 0.13000000, 1634434789.1941, s, ...",...,"[61200.10000, 0.02421483, 1634440513.4426, b, ...","[61200.00000, 0.01473325, 1634440523.3883, s, ...","[61200.00000, 0.02822608, 1634440527.3627, s, ...","[61200.00000, 0.02822608, 1634440534.355, s, l, ]","[61200.00000, 0.02384693, 1634440536.6505, s, ...","[61200.00000, 0.01082407, 1634440538.5282, s, ...","[61200.10000, 0.00010000, 1634440539.0524, b, ...","[61200.00000, 0.00621180, 1634440550.278, s, l, ]","[61200.00000, 0.00483622, 1634440552.3562, s, ...","[61200.00000, 0.00819712, 1634440556.5632, s, ..."
3,"[61200.00000, 0.00070492, 1634440556.5654, s, ...","[61200.00000, 0.00065410, 1634440556.5672, s, ...","[61200.00000, 0.00027817, 1634440556.5687, s, ...","[61200.00000, 0.00040899, 1634440556.57, s, l, ]","[61200.00000, 0.00013086, 1634440556.5722, s, ...","[61200.10000, 0.00060000, 1634440564.6511, b, ...","[61200.10000, 0.00097300, 1634440581.4891, b, ...","[61200.00000, 0.02185432, 1634440589.9378, s, ...","[61200.00000, 0.00140000, 1634440598.7995, s, ...","[61200.00000, 0.01392658, 1634440611.0256, s, ...",...,"[61141.60000, 0.02385947, 1634444788.8149, s, ...","[61141.60000, 0.01414053, 1634444788.8156, s, ...","[61141.60000, 0.03285947, 1634444788.8166, s, ...","[61141.60000, 0.00010000, 1634444788.8172, s, ...","[61141.50000, 0.00010000, 1634444788.8188, s, ...","[61126.2000

In [45]:
# def get_past_points(past_unix): 
    

# resp = requests.get('https://api.kraken.com/0/public/Trades?pair=xbtusd&since=1634429100').json()
# resp['result']


{'XXBTZUSD': [['60752.80000', '0.00240219', 1634429100.1276, 's', 'l', ''],
  ['60751.20000', '0.00113202', 1634429100.1287, 's', 'l', ''],
  ['60751.20000', '0.00060000', 1634429114.6284, 's', 'l', ''],
  ['60750.40000', '0.00333289', 1634429116.0743, 'b', 'm', ''],
  ['60750.40000', '0.00100100', 1634429117.0587, 'b', 'l', ''],
  ['60750.30000', '0.00015499', 1634429119.6577, 's', 'l', ''],
  ['60750.30000', '0.00790151', 1634429119.8435, 's', 'm', ''],
  ['60750.00000', '0.01209849', 1634429119.8445, 's', 'm', ''],
  ['60750.00000', '0.00290151', 1634429146.4614, 's', 'l', ''],
  ['60748.80000', '0.00493837', 1634429146.4628, 's', 'l', ''],
  ['60748.10000', '0.00216012', 1634429146.4637, 's', 'l', ''],
  ['60748.10000', '0.00199208', 1634429147.2826, 's', 'l', ''],
  ['60746.10000', '0.00022857', 1634429147.3709, 's', 'l', ''],
  ['60745.50000', '0.00433864', 1634429147.3718, 's', 'l', ''],
  ['60740.70000', '0.00330000', 1634429147.8671, 's', 'l', ''],
  ['60740.70000', '0.0016390

In [40]:
# used to get the ohlc data but only returns last 720 points of data 
resp = requests.get('https://api.kraken.com/0/public/OHLC?pair=XBTUSD&interval=1&since=1634429100').json()
resp['result']


{'XXBTZUSD': [[1638722940,
   '48690.1',
   '48739.4',
   '48690.0',
   '48728.6',
   '48700.7',
   '1.59714076',
   31],
  [1638723000,
   '48734.2',
   '48734.2',
   '48669.3',
   '48669.3',
   '48729.6',
   '0.66280911',
   16],
  [1638723060,
   '48659.8',
   '48659.8',
   '48631.9',
   '48631.9',
   '48646.2',
   '0.26411834',
   8],
  [1638723120,
   '48631.9',
   '48673.3',
   '48612.0',
   '48666.0',
   '48654.8',
   '0.26791464',
   24],
  [1638723180,
   '48666.0',
   '48666.0',
   '48619.3',
   '48619.4',
   '48628.3',
   '0.54371736',
   29],
  [1638723240,
   '48606.0',
   '48614.0',
   '48601.9',
   '48601.9',
   '48607.5',
   '0.13464013',
   7],
  [1638723300,
   '48601.8',
   '48601.9',
   '48562.5',
   '48562.5',
   '48575.7',
   '0.06315834',
   7],
  [1638723360,
   '48562.1',
   '48651.3',
   '48562.0',
   '48651.3',
   '48602.5',
   '0.58864490',
   36],
  [1638723420,
   '48651.3',
   '48672.5',
   '48640.0',
   '48651.5',
   '48653.6',
   '0.15295358',
   16],
 

TEST CODE BELOW THIS LINE 

In [36]:
#DO not change the below cell. will use this if something breaks and need a fresh copy of the code 

class KrakenWebSocketClient:
    """
    Kraken Client Interface

    """

    def __init__(self, ):
        websocket.enableTrace(False)
        ws = websocket.WebSocketApp("wss://ws.kraken.com/",
                                         on_message=self.on_message,
                                         on_error=self.on_error,
                                         on_close=self.on_close)
        self.ws = ws
        self.ws.on_open = self.on_open
        self.ws.run_forever()

    def on_message(self,ws, message):
        print(message)
        return message

    def on_error(self,ws, error):
        print(error)
        return error

    def on_close(self, a,b,c):
        self.ws.close()
        print("### closed ###")

    def run(self, *args):
        global driver
        driver = True
        while driver:
            try:
                time.sleep(1)
                
            except KeyboardInterrupt:
                driver = False
        time.sleep(1)
        self.ws.close()
        print("thread terminating...")

    def on_open(self,ws):
        self.ws.send('{"event":"subscribe", "subscription":{"name":"ohlc"}, "pair":["XBT/USD"]}')
        t = threading.Thread(target=self.run)
        t.start()
#         thread.start_new_thread(self.run, ())
        



In [10]:
# use this if something reallly breaks.    but would rather use the class implementation 

def ws_thread(class_object):
    ws = websocket.WebSocketApp("wss://ws.kraken.com/", 
                                on_open    = ws_open, 
                                on_error   = ws_error,
                                on_message = partial(ws_message, class_object=class_object)) 
    ws.run_forever()
    
def ws_open(ws):
    ws.send('{"event":"subscribe", "subscription":{"name":"ohlc"}, "pair":["XBT/USD"]}')
def ws_error(ws, error):
    print(error)
def ws_message(ws, message, class_object):
    if "event" not in message: 
        formated_point = json.loads(message)
        print('WebSocket: '+formated_point[1][0]+' '+ formated_point[1][1]+' '+formated_point[1][2]+' '+str(formated_point[1][-1]))
        class_object.to_data_que(formated_point[1])
        

def main():
    BTC =BTC_Data()
    t = threading.Thread(target=ws_thread(BTC))
    t.start()


# if __name__ == '__main__':
#     main()

In [ ]:
# used to get the tradable pairs 
resp = requests.get('https://api.kraken.com/0/public/AssetPairs').json()

testing = pd.DataFrame(resp['result'])
# testing
for x in testing:
    print(testing[x].altname, testing[x].wsname)
# WBTC/USD